In [140]:
# imports 
import numpy as np
import pandas as pd

In [152]:
df= pd.read_csv('../data/train/diamonds_train_cleaned.csv', index_col=[0])
df['xy'] = df['x']/df['y']
df['xz'] = df['x']/df['z']
df['zy'] = df['z']/df['y']
df['table_depth'] = df['table']/df['depth']
#df['xyz'] = df['x']* df['y'] * df['z']
#df['xyz_sum'] = df['x'] + df['y'] + df['z']
df['carat/dimensons'] = df ['carat'] / (df['x']* df['y'] * df['z'])
df.replace([np.inf, -np.inf], 0 , inplace=True)

df.head()

,depth,table,x,y,z,price,carat,cut,color,clarity,city,xy,xz,zy,table_depth
0,62.4,58.0,6.83,6.79,4.25,4268,1.21,Premium,J,VS2,Dubai,1.005891,1.607059,0.625920,0.929487
1,61.6,58.0,6.40,6.35,3.93,3513,1.02,Premium,J,VS2,Dubai,1.007874,1.628499,0.618898,0.941558
2,62.3,58.0,5.86,5.80,3.63,1792,0.77,Premium,J,VS2,Dubai,1.010345,1.614325,0.625862,0.930979
3,59.6,60.0,7.58,7.48,4.49,7553,1.51,Premium,J,VS2,Dubai,1.013369,1.688196,0.600267,1.006711
4,60.2,62.0,5.40,5.33,3.23,1176,0.57,Premium,J,VS2,Dubai,1.013133,1.671827,0.606004,1.029900


In [153]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_columns = list((df.select_dtypes(include=numerics)).columns)
numerical_columns.remove('price')
#numerical_columns.remove('depth')
#numerical_columns.remove('table')
numerical_columns.remove('x')
numerical_columns.remove('y')
numerical_columns.remove('z')
numerical_columns

['depth', 'table', 'carat', 'xy', 'xz', 'zy', 'table_depth']

In [154]:
#numerical_columns = ['carat', 'x', 'y', 'z','xy','xz','zy','xyz',]
categorical_columns = ['cut', 'color', 'clarity']
FEATS = numerical_columns + categorical_columns
TARGET = 'price'


In [155]:
df[FEATS]

,depth,table,carat,xy,xz,zy,table_depth,cut,color,clarity
0,62.4,58.0,1.21,1.005891,1.607059,0.625920,0.929487,Premium,J,VS2
1,61.6,58.0,1.02,1.007874,1.628499,0.618898,0.941558,Premium,J,VS2
2,62.3,58.0,0.77,1.010345,1.614325,0.625862,0.930979,Premium,J,VS2
3,59.6,60.0,1.51,1.013369,1.688196,0.600267,1.006711,Premium,J,VS2
4,60.2,62.0,0.57,1.013133,1.671827,0.606004,1.029900,Premium,J,VS2
...,...,...,...,...,...,...,...,...,...,...
40450,62.2,54.0,0.54,0.994307,1.602446,0.620493,0.868167,Ideal,F,IF
40451,61.9,54.0,0.53,0.994286,1.611111,0.617143,0.872375,Ideal,F,IF
40452,62.3,55.0,0.30,0.990783,1.598513,0.619816,0.882825,Ideal,F,IF
40453,60.9,55.0,0.26,0.981087,1.627451,0.602837,0.903120,Ideal,F,IF


In [156]:
# Apply label encoder to each column with categorical data
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
label_encoder = LabelEncoder()
for col in categorical_columns:
    df[col] = label_encoder.fit_transform(df[col])
df.head()

,depth,table,x,y,z,price,carat,cut,color,clarity,city,xy,xz,zy,table_depth
0,62.4,58.0,6.83,6.79,4.25,4268,1.21,3,6,5,Dubai,1.005891,1.607059,0.625920,0.929487
1,61.6,58.0,6.40,6.35,3.93,3513,1.02,3,6,5,Dubai,1.007874,1.628499,0.618898,0.941558
2,62.3,58.0,5.86,5.80,3.63,1792,0.77,3,6,5,Dubai,1.010345,1.614325,0.625862,0.930979
3,59.6,60.0,7.58,7.48,4.49,7553,1.51,3,6,5,Dubai,1.013369,1.688196,0.600267,1.006711
4,60.2,62.0,5.40,5.33,3.23,1176,0.57,3,6,5,Dubai,1.013133,1.671827,0.606004,1.029900


In [157]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

In [158]:
from sklearn.model_selection import train_test_split

data_train, data_test, target_train, target_test = train_test_split(df[FEATS], df[TARGET], random_state=42)

In [159]:
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor


pipeline_xgb=Pipeline([("scalar5",StandardScaler()),
                     ("rf_classifier",XGBRegressor())]);


In [160]:
pipeline_xgb.fit(data_train, target_train);

In [161]:
y_pred = pipeline_xgb.predict(data_test)
y_pred

array([ 562.0565 , 1322.7809 , 4008.5708 , ..., 2443.6758 ,  464.24808,
        787.3991 ], dtype=float32)

In [162]:
from sklearn.metrics import mean_squared_error
rmse = mean_squared_error(target_test, y_pred)**0.5
rmse

526.5006015512286

In [164]:
#Model prediction Test Dataframe (aka real prediction):
diamonds_train=pd.read_csv('../data/test/diamonds_test.csv', index_col=[0])
diamonds_train['xy'] = diamonds_train['x']/diamonds_train['y']
diamonds_train['xz'] = diamonds_train['x']/diamonds_train['z']
diamonds_train['zy'] = diamonds_train['z']/diamonds_train['y']
diamonds_train['table_depth'] = diamonds_train['table']/df['depth']
#diamonds_train['xyz'] = diamonds_train['x']* diamonds_train['y'] * diamonds_train['z']
#diamonds_train['xyz_sum'] = diamonds_train['x'] + diamonds_train['y'] + diamonds_train['z']
diamonds_train['carat/dimensons'] = diamonds_train ['carat'] / (diamonds_train['x']* diamonds_train['y'] * diamonds_train['z'])
diamonds_train.replace([np.inf, -np.inf], 0 , inplace=True)
diamonds_train


,carat,cut,color,clarity,depth,table,x,y,z,city,xy,xz,zy,table_depth,carat/dimensons
id,,,,,,,,,,,,,,,
0,0.79,Very Good,F,SI1,62.7,60.0,5.82,5.89,3.67,Amsterdam,0.988115,1.585831,0.623090,0.961538,0.006279
1,1.20,Ideal,J,VS1,61.0,57.0,6.81,6.89,4.18,Surat,0.988389,1.629187,0.606676,0.925325,0.006118
2,1.57,Premium,H,SI1,62.2,61.0,7.38,7.32,4.57,Kimberly,1.008197,1.614880,0.624317,0.979133,0.006359
3,0.90,Very Good,F,SI1,63.8,54.0,6.09,6.13,3.90,Kimberly,0.993475,1.561538,0.636215,0.906040,0.006182
4,0.50,Very Good,F,VS1,62.9,58.0,5.05,5.09,3.19,Amsterdam,0.992141,1.583072,0.626719,0.963455,0.006098
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13480,0.57,Ideal,E,SI1,61.9,56.0,5.35,5.32,3.30,Amsterdam,1.005639,1.621212,0.620301,0.919540,0.006069
13481,0.71,Ideal,I,VS2,62.2,55.0,5.71,5.73,3.56,New York City,0.996510,1.603933,0.621291,0.878594,0.006096
13482,0.70,Ideal,F,VS1,61.6,55.0,5.75,5.71,3.53,Tel Aviv,1.007005,1.628895,0.618214,0.892857,0.006040


In [165]:
for col in categorical_columns:
    diamonds_train[col] = label_encoder.fit_transform(diamonds_train[col])
diamonds_train.head()

,carat,cut,color,clarity,depth,table,x,y,z,city,xy,xz,zy,table_depth,carat/dimensons
id,,,,,,,,,,,,,,,
0,0.79,4,2,2,62.7,60.0,5.82,5.89,3.67,Amsterdam,0.988115,1.585831,0.623090,0.961538,0.006279
1,1.20,2,6,4,61.0,57.0,6.81,6.89,4.18,Surat,0.988389,1.629187,0.606676,0.925325,0.006118
2,1.57,3,4,2,62.2,61.0,7.38,7.32,4.57,Kimberly,1.008197,1.614880,0.624317,0.979133,0.006359
3,0.90,4,2,2,63.8,54.0,6.09,6.13,3.90,Kimberly,0.993475,1.561538,0.636215,0.906040,0.006182
4,0.50,4,2,4,62.9,58.0,5.05,5.09,3.19,Amsterdam,0.992141,1.583072,0.626719,0.963455,0.006098


In [166]:
pipeline_xgb.fit(df[FEATS],df[TARGET]); #Volvemos a entrenar el modelo con el datframe original (sin la separacion de train y test), para tener mas datos en el entreno

In [167]:
diamonds_train[FEATS]

,depth,table,carat,xy,xz,zy,table_depth,cut,color,clarity
id,,,,,,,,,,
0,62.7,60.0,0.79,0.988115,1.585831,0.623090,0.961538,4,2,2
1,61.0,57.0,1.20,0.988389,1.629187,0.606676,0.925325,2,6,4
2,62.2,61.0,1.57,1.008197,1.614880,0.624317,0.979133,3,4,2
3,63.8,54.0,0.90,0.993475,1.561538,0.636215,0.906040,4,2,2
4,62.9,58.0,0.50,0.992141,1.583072,0.626719,0.963455,4,2,4
...,...,...,...,...,...,...,...,...,...,...
13480,61.9,56.0,0.57,1.005639,1.621212,0.620301,0.919540,2,1,2
13481,62.2,55.0,0.71,0.996510,1.603933,0.621291,0.878594,2,5,5
13482,61.6,55.0,0.70,1.007005,1.628895,0.618214,0.892857,2,2,4


In [169]:
y_pred = pipeline_xgb.predict(diamonds_train[FEATS])
y_pred


array([2831.072 , 5678.7856, 9374.825 , ..., 3059.4316, 2089.6006,
        767.8283], dtype=float32)

In [171]:
y_pred=pd.DataFrame(y_pred, columns=['price'])
y_pred.reset_index(inplace=True)
y_pred.rename(columns={"index": "id"}, inplace= True)
y_pred

,id,price
0,0,2831.072021
1,1,5678.785645
2,2,9374.825195
3,3,4031.762207
4,4,1620.834351
...,...,...
13480,13480,1616.020996
13481,13481,2489.738770
13482,13482,3059.431641
13483,13483,2089.600586


In [172]:
y_pred.to_csv('../data/prediction/price_prediction.csv', index= False)